In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

## Loading Data

In [ ]:
saved_data = np.load('processed_data.npz')
X = saved_data['X'][:170,:,:]
X_sym = saved_data['X_sym'][:170,:,:]
Y = saved_data['Y'][:170,:]
Z = saved_data['Z'][:170,:]
pids = saved_data['pids'][:170]

M = X.shape[0]
p = X.shape[1]; q = Y.shape[1]

kf = KFold(n_splits=10)

## Stability of PCA Components (10-fold CV)

In [ ]:
kf = KFold(n_splits=10)
comp = np.zeros((10,81,54756))
c = 0
for train_index, test_index in kf.split(X):
    x_train = X[train_index,:,0]
    x_test = X[test_index,:,0]
    
    pca = PCA(n_components=81)
    pca.fit_transform(x_train)
    comp[c,:,:] = pca.components_
    c += 1

sim = np.zeros((45,81))
c = 0
for i in range(10):
    for j in range(i+1,10):
        sim[c,:] = np.diag(np.abs(comp[i,:,:] @ comp[j,:,:].T))
        c += 1

var = np.var(sim, axis=0)
plt.figure()
plt.plot(var)

## Stability of PCA Components of Symmetric Data Matrix (10-fold CV)

In [ ]:
comp = np.zeros((10,81,27495))
c = 0
for train_index, test_index in kf.split(X_sym):
    x_train = X_sym[train_index,:,0]
    x_test = X_sym[test_index,:,0]
    
    pca = PCA(n_components=81)
    pca.fit_transform(x_train)
    comp[c,:,:] = pca.components_
    c += 1

sim = np.zeros((45,81))
c = 0
for i in range(10):
    for j in range(i+1,10):
        sim[c,:] = np.diag(np.abs(comp[i,:,:] @ comp[j,:,:].T))
        c += 1

var = np.var(sim, axis=0)
plt.figure()
plt.plot(var)

## Mean/Variance of Normalized Error from PCA of X Over 10-fold CV

In [ ]:
train_err = np.zeros((10,80))
test_err = np.zeros((10,80))

s = 0
for train_index, test_index in kf.split(X):
    x_train = X[train_index,:,0]
    x_test = X[test_index,:,0]
    train_norms = np.linalg.norm(x_train, axis=1)
    test_norms = np.linalg.norm(x_test, axis=1)
    pca = PCA(n_components=81)
    pca.fit_transform(x_train)
    components = pca.components_
    for d in range(1,81):
        comp = components[:d,:]
        xh_train = x_train @ comp.T @ comp
        xh_test = x_test @ comp.T @ comp
        train_err[s,d-1] = np.mean(np.linalg.norm(xh_train - x_train, axis=1) / train_norms) # error normalized by magnitude of data
        test_err[s,d-1] = np.mean(np.linalg.norm(xh_test - x_test, axis=1) / test_norms) # error normalized by magnitude of data
    s += 1


plt.figure()
plt.plot(np.mean(train_err,axis=0)); plt.plot(np.mean(test_err,axis=0))
plt.figure()
plt.plot(np.var(train_err,axis=0)); plt.plot(np.var(test_err,axis=0))

## Mean/Variance of Normalized Error from PCA of Y Over 10-fold CV

In [ ]:
y_train_err = np.zeros((10,40))
y_test_err = np.zeros((10,40))
y_explained_var = np.zeros((10,41))

s = 0
for train_index, test_index in kf.split(Y):
    y_train = Y[train_index,:] + 1
    y_test = Y[test_index,:] + 1
    train_norms = np.linalg.norm(y_train, axis=1)
    test_norms = np.linalg.norm(y_test, axis=1)
    pca = PCA(n_components=41,whiten=False)
    pca.fit_transform(y_train)
    components = pca.components_
#     y_explained_var[s,:] = pca.explained_variance_ratio_
    for d in range(1,41):
        comp = components[:d,:]
        yh_train = y_train @ comp.T @ comp
        yh_test = y_test @ comp.T @ comp
        y_explained_var[s,d] = np.sum(pca.explained_variance_ratio_[:d])
        y_train_err[s,d-1] = np.mean(np.linalg.norm(yh_train - y_train, axis=1) / train_norms)
        y_test_err[s,d-1] = np.mean(np.linalg.norm(yh_test - y_test, axis=1) / test_norms)
    s += 1


plt.figure()
plt.plot(np.mean(y_explained_var,axis=0)); plt.grid()
plt.figure()
plt.plot(np.mean(y_train_err,axis=0)); 
plt.plot(np.mean(y_test_err,axis=0))
plt.figure()
plt.plot(np.var(y_train_err,axis=0)); 
plt.plot(np.var(y_test_err,axis=0))

## Mean/Variance of Normalized Error from PCA of Z Over 10-fold CV

In [ ]:
z_train_err = np.zeros((10,29))
z_test_err = np.zeros((10,29))
z_explained_var = np.zeros((10,30))

s = 0
for train_index, test_index in kf.split(Z):
    z_train = Z[train_index,:] + 1
    z_test = Z[test_index,:] + 1
    train_norms = np.linalg.norm(z_train, axis=1)
    test_norms = np.linalg.norm(z_test, axis=1)
    pca = PCA(n_components=30,whiten=False)
    pca.fit_transform(z_train)
    components = pca.components_
#     z_explained_var[s,:] = pca.explained_variance_ratio_
    for d in range(1,30):
        comp = components[:d,:]
        zh_train = z_train @ comp.T @ comp
        zh_test = z_test @ comp.T @ comp
        z_explained_var[s,d] = np.sum(pca.explained_variance_ratio_[:d])
        z_train_err[s,d-1] = np.mean(np.linalg.norm(zh_train - z_train, axis=1) / train_norms)
        z_test_err[s,d-1] = np.mean(np.linalg.norm(zh_test - z_test, axis=1) / test_norms)
    s += 1


plt.figure()
plt.plot(np.mean(z_explained_var,axis=0)); plt.grid()
plt.figure()
plt.plot(np.mean(z_train_err,axis=0)); 
plt.plot(np.mean(z_test_err,axis=0))
plt.figure()
plt.plot(np.var(z_train_err,axis=0)); 
plt.plot(np.var(z_test_err,axis=0))

## Predicting Q1 Using 81-dimensions of X (10-fold CV)

In [ ]:
train_err = np.zeros((5,))
test_err = np.zeros((5,))

s = 0
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    x_train = X[train_index,:,0]
    x_test = X[test_index,:,0]
#     pca = PCA(n_components=81)
#     xh_train = pca.fit_transform(x_train)
#     xh_test = pca.transform(x_test)
    y_train = Y[train_index,0]; y_test = Y[test_index,0]
    clf = LogisticRegression(solver='lbfgs', multi_class='multinomial').fit(x_train, y_train)
    yh_train = clf.predict(x_train)
    yh_test = clf.predict(x_test)

    train_err[s] = 1 - sum(y_train == yh_train) / len(y_train)
    test_err[s] = 1 - sum(y_test == yh_test) / len(y_test)
    
    s += 1

print("Mean Training Error: %.3f" % np.mean(train_err))
print("Mean Test Error: %.3f" % np.mean(test_err))
print("Std Training Error: %.3f" % np.std(train_err))
print("Std Test Error: %.3f" % np.std(test_err))
plt.figure()
plt.hist(train_err);
plt.figure()
plt.hist(test_err);

## Predicting All Questions Using 81-dimensions of X (10-fold CV)

In [ ]:
train_err = np.zeros((5,41))
test_err = np.zeros((5,41))

s = 0
for train_index, test_index in kf.split(X):
    x_train = X[train_index,:,0]
    x_test = X[test_index,:,0]
    pca = PCA(n_components=81)
    xh_train = pca.fit_transform(x_train)
    xh_test = pca.transform(x_test)
    for q in range(41):
        y_train = Y[train_index,q]; y_test = Y[test_index,q]
        clf = LogisticRegression(solver='lbfgs', multi_class='multinomial').fit(xh_train, y_train)
        yh_train = clf.predict(xh_train)
        yh_test = clf.predict(xh_test)

        train_err[s,q] = 1 - sum(y_train == yh_train) / 153
        test_err[s,q] = 1 - sum(y_test == yh_test) / 17
    
    s += 1

print("Mean Training Error Across All Questions: %.3f" % np.mean(train_err))
print("Mean Test Error Across All Questions: %.3f" % np.mean(test_err))
plt.figure()
plt.hist(np.mean(train_err,axis=0));
plt.figure()
plt.hist(np.mean(test_err,axis=0));

## Distribution of Questionnaire Data

In [ ]:
plt.figure()
plt.hist(Y.flatten())
plt.figure()
plt.hist(Z.flatten())